# Setup

In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from datasets import load_dataset
from pydantic import BaseModel
from dotenv import load_dotenv
from loguru import logger
from datetime import timedelta
import sys
import dask.dataframe as dd
from sqlalchemy.types import Text, ARRAY, JSON
import gc

sys.path.insert(0, "..")
sys.path.insert(1, "../../")

from src.utils.data_prep import parse_dt, handle_dtypes
from src.utils.data_prep import insert_chunk_to_oltp
import os

In [3]:
_ = load_dotenv(override=True)

## Arguments

In [4]:
class Args(BaseModel):
    random_seed: int = 41
    testing: bool = False

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    
    train_fp: str = "../../data_for_ai/interim/train_sample_interactions_16407u.parquet"
    val_fp: str = "../../data_for_ai/interim/val_sample_interactions_16407u.parquet"

    transaction_table_name: str = "amz_rating"
    metadata_table_name: str = "amz_metadata"

    hf_dataset_name: str = "McAuley-Lab/Amazon-Reviews-2023"
    amz_metadata_hf_dataset_path: str = "raw_meta_Electronics"

    # Number of days left out not being pushed into the OLTP so that later we can simulate having them as new data
    num_days_holdout: int = 30
    holdout_fp: str = "../../data_for_ai/experimental/holdout.parquet"

    data_persist_dir: str = None

    def init(self):
        if not self.testing:
            self.data_persist_dir = os.path.abspath("../../data_for_ai/raw")   
            os.makedirs(self.data_persist_dir, exist_ok=True)
        return self

args = Args().init()
print(args.model_dump_json(indent=2))


{
  "random_seed": 41,
  "testing": false,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "train_fp": "../../data_for_ai/interim/train_sample_interactions_16407u.parquet",
  "val_fp": "../../data_for_ai/interim/val_sample_interactions_16407u.parquet",
  "transaction_table_name": "amz_rating",
  "metadata_table_name": "amz_metadata",
  "hf_dataset_name": "McAuley-Lab/Amazon-Reviews-2023",
  "amz_metadata_hf_dataset_path": "raw_meta_Electronics",
  "num_days_holdout": 30,
  "holdout_fp": "../../data_for_ai/experimental/holdout.parquet",
  "data_persist_dir": "/home/dinhln/Desktop/real_time_recsys/data_for_ai/raw"
}


## Load dataset

In [5]:
amz_metadata_raw = load_dataset(args.hf_dataset_name,
                                args.amz_metadata_hf_dataset_path,
                                split="full",
                                trust_remote_code=True)

In [6]:
amz_metadata_df = amz_metadata_raw.to_pandas()
amz_metadata_df.drop_duplicates(subset=[args.item_col], inplace=True)

logger.info(f"amz_metadata_df.shape: {amz_metadata_df.shape}")

2025-06-22 15:04:56.047 | INFO     | __main__:<module>:4 - amz_metadata_df.shape: (1610012, 16)


In [7]:
with pd.option_context('display.max_colwidth',100):
    display(amz_metadata_df.head(5))

main_category  \
0            All Electronics   
1            All Electronics   
2                  Computers   
3             AMAZON FASHION   
4  Cell Phones & Accessories   

                                                                                                 title  \
0                                                               FS-1051 FATSHARK TELEPORTER V3 HEADSET   
1                                                                        Ce-H22B12-S1 4Kx2K Hdmi 4Port   
2  Digi-Tatoo Decal Skin Compatible With MacBook Pro 13 inch (Model A2338/ A2289/ A2251) - Protecti...   
3  NotoCity Compatible with Vivoactive 4 band 22mm Quick Release Silicone Bands/Garmin Darth Vader/...   
4                                                               Motorola Droid X Essentials Combo Pack   

   average_rating  rating_number  \
0             3.5              6   
1             5.0              1   
2             4.5            246   
3             4.5            233   
4             3.8             64   

                                                                                              features  \
0                                                                                                   []   
1                                                               [UPC: 662774021904, Weight: 0.600 lbs]   
2  [WARNING: Please IDENTIFY MODEL NUMBER on the bottom of your Macbook. Only fits for model A2338/...   
3  [☛NotoCity 22mm band is designed for Vivoactive 4 / Samsung Gear S3 Classic / S3 Frontier / Gear...   
4  [New Droid X Essentials Combo Pack, Exclusive Package Incredible Value Worth $145!!!, Includes a...   

                                                                                           description  \
0  [Teleporter V3 The “Teleporter V3” kit sets a new level of value in the FPV world with Fat Shark...   
1                                                                                 [HDMI In - HDMI Out]   
2                                                                                                   []   
3                                                                                                   []   
4  [all Genuine High Quality Motorola Made Accessories, including Multimedia Station with HDMI tech...   

   price  \
0   None   
1   None   
2  19.99   
3   9.99   
4  14.99   

                                                                                                images  \
0  {'hi_res': [None], 'large': ['https://m.media-amazon.com/images/I/41qrX56lsYL._AC_.jpg'], 'thumb...   
1  {'hi_res': ['https://m.media-amazon.com/images/I/51qxU4Zd5TL._AC_SL1050_.jpg', 'https://m.media-...   
2  {'hi_res': ['https://m.media-amazon.com/images/I/61RPxmi+mPL._AC_SL1500_.jpg', 'https://m.media-...   
3  {'hi_res': ['https://m.media-amazon.com/images/I/51ajKKbi76L._AC_UL1000_.jpg', 'https://m.media-...   
4  {'hi_res': [None, None, None, None, None], 'large': ['https://m.media-amazon.com/images/I/51-DXS...   

                                                                                                videos  \
0                                                              {'title': [], 'url': [], 'user_id': []}   
1                                                              {'title': [], 'url': [], 'user_id': []}   
2  {'title': ['AL 2Sides Video', 'MacBook Protective Skin', 'ARTSO 15.6" Laptop Skin Sticker Univer...   
3                                                              {'title': [], 'url': [], 'user_id': []}   
4                                                              {'title': [], 'url': [], 'user_id': []}   

        store  \
0   Fat Shark   
1        SIIG   
2  Digi-Tatoo   
3    NotoCity   
4     Verizon   

                                                                                            categories  \
0                                                     [Electronics, Television & Video, Video Glasses]   
1                               

In [8]:
cols = ["parent_asin", "main_category", "title", "description", "price", "categories", "details"]
metadata_df = amz_metadata_df[cols]
metadata_df.head(5)

parent_asin              main_category  \
0  B00MCW7G9M            All Electronics   
1  B00YT6XQSE            All Electronics   
2  B07SM135LS                  Computers   
3  B089CNGZCW             AMAZON FASHION   
4  B004E2Z88O  Cell Phones & Accessories   

                                               title  \
0             FS-1051 FATSHARK TELEPORTER V3 HEADSET   
1                      Ce-H22B12-S1 4Kx2K Hdmi 4Port   
2  Digi-Tatoo Decal Skin Compatible With MacBook ...   
3  NotoCity Compatible with Vivoactive 4 band 22m...   
4             Motorola Droid X Essentials Combo Pack   

                                         description  price  \
0  [Teleporter V3 The “Teleporter V3” kit sets a ...   None   
1                               [HDMI In - HDMI Out]   None   
2                                                 []  19.99   
3                                                 []   9.99   
4  [all Genuine High Quality Motorola Made Access...  14.99   

                                          categories  \
0   [Electronics, Television & Video, Video Glasses]   
1  [Electronics, Television & Video, Accessories,...   
2  [Electronics, Computers & Accessories, Laptop ...   
3  [Electronics, Wearable Technology, Clips, Arm ...   
4  [Electronics, Computers & Accessories, Compute...   

                                             details  
0  {"Date First Available": "August 2, 2014", "Ma...  
1  {"Product Dimensions": "0.83 x 4.17 x 2.05 inc...  
2  {"Brand": "Digi-Tatoo", "Color": "Fresh Marble...  
3  {"Date First Available": "May 29, 2020", "Manu...  
4  {"Product Dimensions": "11.6 x 6.9 x 3.1 inche...

In [9]:
amz_metadata_df[args.item_col].nunique()

1610012

### Get rating sample

In [10]:
if not os.path.exists(args.train_fp):
    raise Exception(
        f"{args.train_fp} does not exist, you need to run the notebook 000-prep-data in the parent folder first"
    )

train_df = pd.read_parquet(args.train_fp)
val_df = pd.read_parquet(args.val_fp)

In [11]:
train_df

user_id parent_asin  rating           timestamp
3194      AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B06XKCPK5W     2.0 2012-06-11 16:41:10
3199      AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B000CKVOOY     3.0 2012-08-02 02:04:13
3200      AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B006GWO5WK     5.0 2012-09-15 16:34:46
3204      AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B008LURQ76     5.0 2013-01-03 23:08:45
3208      AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B00AQRUW4Q     4.0 2013-05-06 01:24:39
...                                ...         ...     ...                 ...
40882304  AFB4DWWKZBQFS22FAWDEP37EL2FA  B00KAF5RQ2     5.0 2016-02-22 17:44:10
40882305  AFB4DWWKZBQFS22FAWDEP37EL2FA  B001F6TXME     5.0 2016-02-22 17:44:40
40882306  AFB4DWWKZBQFS22FAWDEP37EL2FA  B007VGGIB6     5.0 2016-02-22 17:45:10
40882307  AFB4DWWKZBQFS22FAWDEP37EL2FA  B00WUID73W     5.0 2016-02-22 17:45:37
40882310  AFB4DWWKZBQFS22FAWDEP37EL2FA  B00SFSU53G     5.0 2016-07-15 00:58:19

[127392 rows x 4 columns]

## Megre metadata

In [12]:
cols = ["main_category", "title", "description", "categories", "price"]

# Merge the item features into the interaction data
train_features_df = pd.merge(
    train_df, amz_metadata_df[[args.item_col] + cols], how="left", on=args.item_col
)
val_features_df = pd.merge(
    val_df, amz_metadata_df[[args.item_col] + cols], how="left", on=args.item_col
)
train_features_df

user_id parent_asin  rating           timestamp  \
0       AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B06XKCPK5W     2.0 2012-06-11 16:41:10   
1       AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B000CKVOOY     3.0 2012-08-02 02:04:13   
2       AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B006GWO5WK     5.0 2012-09-15 16:34:46   
3       AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B008LURQ76     5.0 2013-01-03 23:08:45   
4       AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B00AQRUW4Q     4.0 2013-05-06 01:24:39   
...                              ...         ...     ...                 ...   
127387  AFB4DWWKZBQFS22FAWDEP37EL2FA  B00KAF5RQ2     5.0 2016-02-22 17:44:10   
127388  AFB4DWWKZBQFS22FAWDEP37EL2FA  B001F6TXME     5.0 2016-02-22 17:44:40   
127389  AFB4DWWKZBQFS22FAWDEP37EL2FA  B007VGGIB6     5.0 2016-02-22 17:45:10   
127390  AFB4DWWKZBQFS22FAWDEP37EL2FA  B00WUID73W     5.0 2016-02-22 17:45:37   
127391  AFB4DWWKZBQFS22FAWDEP37EL2FA  B00SFSU53G     5.0 2016-07-15 00:58:19   

          main_category                                              title  \
0             Computers  Targus CleanVu Cleaning Pads Touch Screen Devi...   
1             Computers  Arkon Folding Tablet Stand Compatible with App...   
2        Amazon Devices  Amazon Kindle 9W PowerFast Adapter for Acceler...   
3             Computers  amFilm (TM) Premium Screen Protector Film Clea...   
4             Computers  Seagate Wireless Plus 1TB Portable Hard Drive ...   
...                 ...                                                ...   
127387   Camera & Photo  STK EN-EL14 EN-EL14a Battery Pack for Nikon D3...   
127388   Camera & Photo   Nikon 25395 MC-DC2 Remote Release Cord (1 Meter)   
127389   Camera & Photo  Nikon WU-1a Wireless Mobile Adapter 27081 for ...   
127390   Camera & Photo  eForCity Leather Hand Grip Strap Compatible wi...   
127391  All Electronics  Mounting Dream TV Wall Mount for 32-65 Inch TV...   

                                              description  \
0       [The Targus CleanVu Cleaning Pads for iPad off...   
1       [IPM-TAB1 is a highly versatile folding tablet...   
2                                                      []   
3       [Introducing amFilm® Premium Screen Protector ...   
4       [With Seagate Wireless Plus, you can enjoy you...   
...                                                   ...   
127387  [The STK Nikon EN-EL14 battery is 100% compati...   
127388  [Nikon MC-DC2 Remote Release Cord for Nikon Di...   
127389  [The WU-1a is compatible with the following Ni...   
127390  [Product Description Cushioned camera grips fi...   
127391                                                 []   

                                               categories  price  
0       [Electronics, Television & Video, Accessories,...   6.99  
1       [Electronics, Computers & Accessories, Tablet ...  14.95  
2       [Electronics, Computers & Accessories, Tablet ...   None  
3       [Electronics, Computers & Accessories, Tablet ...   None  
4       [Electronics, Computers & Accessories, Data St...   None  
...                                                   ...    ...  
127387  [Electronics, Camera & Photo, Accessories, Bat...  14.99  
127388  [Electronics, Camera & Photo, Accessories, Dig...  26.95  
127389  [Electronics, Camera & Photo, Accessories, Bat...  98.89  
127390  [Electronics, Camera & Photo, Bags & Cases, Bi...   None  
127391  [Electronics, Television & Video, Accessories,...  47.99  

[127392 rows x 9 columns]

In [13]:
full_df = (
    pd.concat(
        [train_features_df, val_features_df],
        axis=0,
    )
    .assign(
        description=lambda df: df["description"].apply(list),
        categories=lambda df: df["categories"].apply(list),
    )
    .pipe(handle_dtypes)
)
full_df

user_id parent_asin  rating  \
0     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B06XKCPK5W     2.0   
1     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B000CKVOOY     3.0   
2     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B006GWO5WK     5.0   
3     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B008LURQ76     5.0   
4     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B00AQRUW4Q     4.0   
...                            ...         ...     ...   
3474  AHIIISHZP6YAVVHMDEBLJ5CWZ7ZA  B0BZ62FQ13     3.0   
3475  AFTE3G43QHXWD3DJGDCI2DHEWQJQ  B08DMXDPW5     5.0   
3476  AFENZZDPVUYFVBS47YDOWJCDYBSQ  B09XBT6DS9     4.0   
3477  AFMBZYPDAXT5VO3ME67HW5Q5TAOQ  B097KBF8JK     5.0   
3478  AEMSEOMPBE6KWOZWCX4VE4E2E7HQ  B00FB45USW     5.0   

                   timestamp              main_category  \
0    2012-06-11 16:41:10.000                  Computers   
1    2012-08-02 02:04:13.000                  Computers   
2    2012-09-15 16:34:46.000             Amazon Devices   
3    2013-01-03 23:08:45.000                  Computers   
4    2013-05-06 01:24:39.000                  Computers   
...                      ...                        ...   
3474 2021-07-16 17:08:55.044  Cell Phones & Accessories   
3475 2021-01-14 01:48:09.423             Amazon Devices   
3476 2021-12-05 00:35:40.874       Home Audio & Theater   
3477 2022-02-18 11:32:46.732                  Computers   
3478 2021-08-12 00:42:48.843                  Computers   

                                                  title  \
0     Targus CleanVu Cleaning Pads Touch Screen Devi...   
1     Arkon Folding Tablet Stand Compatible with App...   
2     Amazon Kindle 9W PowerFast Adapter for Acceler...   
3     amFilm (TM) Premium Screen Protector Film Clea...   
4     Seagate Wireless Plus 1TB Portable Hard Drive ...   
...                                                 ...   
3474  OtterBox iPhone SE 3rd & 2nd Gen, iPhone 8 & i...   
3475  Echo (4th Gen) | Glacier White with Sengled Bl...   
3476  Ultra Clarity Cables High Speed HDMI Extension...   
3477  Corsair Vengeance LPX 128GB (4x32GB) DDR4 3200...   
3478  Asus (USB-AC56) Dual-band Wireless-AC1300 USB ...   

                                            description  \
0     [The Targus CleanVu Cleaning Pads for iPad off...   
1     [IPM-TAB1 is a highly versatile folding tablet...   
2                                                    []   
3     [Introducing amFilm® Premium Screen Protector ...   
4     [With Seagate Wireless Plus, you can enjoy you...   
...                                                 ...   
3474  [Commuter Series is the compact case that keep...   
3475                                                 []   
3476                                                 []   
3477                            [Corsair Vengeance LPX]   
3478  [A Stable And Reliable Connection Whether you’...   

                                             categories   price  
0     [Electronics, Television & Video, Accessories,...    6.99  
1     [Electronics, Computers & Accessories, Tablet ...   14.95  
2     [Electronics, Computers & Accessories, Tablet ...    None  
3     [Electronics, Computers & Accessories, Tablet ...    None  
4     [Electronics, Computers & Accessories, Data St...    None  
...                                                 ...     ...  
3474  [Electronics, Electronics Gift Guide - Top Tec...    None  
3475                                                 []  114.98  
3476  [Electronics, Television & Video, Accessories,...   10.99  
3477  [Electronics, Computers & Accessories, Compute...  279.99  
3478  [Electronics, Computers & Accessories, Network...    None  

[130871 rows x 9 columns]

In [14]:
import plotly.express as px
px.bar(
    data_frame=full_df.groupby(pd.Grouper(key="timestamp", freq="D"))
    .size()
    .reset_index(name="count"),
    x="timestamp",
    y="count",
    title="Number of data points over time",
)

## Get holdout in order to demo data pipeline

In [15]:
holdout_min_date = full_df[args.timestamp_col].max() - timedelta(days=args.num_days_holdout)

logger.info(f"The original shape of full_df: {full_df.shape}")
full_df_train = full_df.loc[lambda df: df[args.timestamp_col] < holdout_min_date]
logger.info(f"The shape of full_df after splitted: {full_df_train.shape}")

holdout_rating_df = full_df.loc[lambda df: df[args.timestamp_col] >= holdout_min_date]
logger.info(f"The shape of holdout_rating_df: {holdout_rating_df.shape}")

assert len(full_df_train) + len(holdout_rating_df) == len(full_df), "Total length of 2 subsets should be equal to the original length"

2025-06-22 15:05:00.000 | INFO     | __main__:<module>:3 - The original shape of full_df: (130871, 9)
2025-06-22 15:05:00.017 | INFO     | __main__:<module>:5 - The shape of full_df after splitted: (130658, 9)
2025-06-22 15:05:00.020 | INFO     | __main__:<module>:8 - The shape of holdout_rating_df: (213, 9)


In [16]:
full_df_train

user_id parent_asin  rating  \
0     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B06XKCPK5W     2.0   
1     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B000CKVOOY     3.0   
2     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B006GWO5WK     5.0   
3     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B008LURQ76     5.0   
4     AEYGPUCRKH7G4VM22FM3VAKSQ23Q  B00AQRUW4Q     4.0   
...                            ...         ...     ...   
3473  AE2GH2QNHK6AVGUOTECE6FOECE2A  B093H9PY4L     5.0   
3474  AHIIISHZP6YAVVHMDEBLJ5CWZ7ZA  B0BZ62FQ13     3.0   
3475  AFTE3G43QHXWD3DJGDCI2DHEWQJQ  B08DMXDPW5     5.0   
3476  AFENZZDPVUYFVBS47YDOWJCDYBSQ  B09XBT6DS9     4.0   
3478  AEMSEOMPBE6KWOZWCX4VE4E2E7HQ  B00FB45USW     5.0   

                   timestamp              main_category  \
0    2012-06-11 16:41:10.000                  Computers   
1    2012-08-02 02:04:13.000                  Computers   
2    2012-09-15 16:34:46.000             Amazon Devices   
3    2013-01-03 23:08:45.000                  Computers   
4    2013-05-06 01:24:39.000                  Computers   
...                      ...                        ...   
3473 2021-12-03 18:03:38.676            All Electronics   
3474 2021-07-16 17:08:55.044  Cell Phones & Accessories   
3475 2021-01-14 01:48:09.423             Amazon Devices   
3476 2021-12-05 00:35:40.874       Home Audio & Theater   
3478 2021-08-12 00:42:48.843                  Computers   

                                                  title  \
0     Targus CleanVu Cleaning Pads Touch Screen Devi...   
1     Arkon Folding Tablet Stand Compatible with App...   
2     Amazon Kindle 9W PowerFast Adapter for Acceler...   
3     amFilm (TM) Premium Screen Protector Film Clea...   
4     Seagate Wireless Plus 1TB Portable Hard Drive ...   
...                                                 ...   
3473  IVANKY 4K HDMI Cable 3.3 ft, High Speed 18Gbps...   
3474  OtterBox iPhone SE 3rd & 2nd Gen, iPhone 8 & i...   
3475  Echo (4th Gen) | Glacier White with Sengled Bl...   
3476  Ultra Clarity Cables High Speed HDMI Extension...   
3478  Asus (USB-AC56) Dual-band Wireless-AC1300 USB ...   

                                            description  \
0     [The Targus CleanVu Cleaning Pads for iPad off...   
1     [IPM-TAB1 is a highly versatile folding tablet...   
2                                                    []   
3     [Introducing amFilm® Premium Screen Protector ...   
4     [With Seagate Wireless Plus, you can enjoy you...   
...                                                 ...   
3473                                                 []   
3474  [Commuter Series is the compact case that keep...   
3475                                                 []   
3476                                                 []   
3478  [A Stable And Reliable Connection Whether you’...   

                                             categories   price  
0     [Electronics, Television & Video, Accessories,...    6.99  
1     [Electronics, Computers & Accessories, Tablet ...   14.95  
2     [Electronics, Computers & Accessories, Tablet ...    None  
3     [Electronics, Computers & Accessories, Tablet ...    None  
4     [Electronics, Computers & Accessories, Data St...    None  
...                                                 ...     ...  
3473  [Electronics, Television & Video, Accessories,...    5.99  
3474  [Electronics, Electronics Gift Guide - Top Tec...    None  
3475                                                 []  114.98  
3476  [Electronics, Television & Video, Accessories,...   10.99  
3478  [Electronics, Computers & Accessories, Network...    None  

[130658 rows x 9 columns]

In [17]:
holdout_rating_df

user_id parent_asin  rating  \
49    AEMV4AQMTRHICJECLPLTC6SV6D3A  B00V7SFR8Y     5.0   
50    AEMV4AQMTRHICJECLPLTC6SV6D3A  B077V2BF3C     5.0   
51    AEMV4AQMTRHICJECLPLTC6SV6D3A  B0C556BW5M     5.0   
52    AEMV4AQMTRHICJECLPLTC6SV6D3A  B0921LXGF9     5.0   
67    AG2CZRZGOVDCD5BBMCGA7CSTGQIQ  B07FP8VLXS     3.0   
...                            ...         ...     ...   
3419  AGFRYVIF7CVPOK777KN3PSOSWSMA  B0BVZZ36ZL     5.0   
3433  AFCCJP56LNBAKFMEOVL36F3SRCKQ  B0B2DD1SC2     1.0   
3435  AFFGZQBIHKANJVKYMYZ4DC7D3HDQ  B09Y3H915M     3.0   
3456  AHA3P7VTPBXCMB22DPNLCKTCIUWQ  B01DN7OJ30     5.0   
3477  AFMBZYPDAXT5VO3ME67HW5Q5TAOQ  B097KBF8JK     5.0   

                   timestamp        main_category  \
49   2022-02-05 02:09:34.769      All Electronics   
50   2022-02-05 02:13:37.521            Computers   
51   2022-02-05 03:17:34.026      All Electronics   
52   2022-02-05 03:37:03.183            Computers   
67   2022-01-21 01:20:36.518            Computers   
...                      ...                  ...   
3419 2022-01-24 19:12:44.117      All Electronics   
3433 2022-02-15 19:28:11.361            Computers   
3435 2022-01-29 19:14:46.007      All Electronics   
3456 2022-01-23 18:55:32.961  Musical Instruments   
3477 2022-02-18 11:32:46.732            Computers   

                                                  title  \
49    VCE HDMI Coupler HDMI Female to Female Connect...   
50    Moread HDMI to VGA, 2 Pack, Gold-Plated HDMI t...   
51    External CD/DVD Drive for Laptop USB 3.0 CD/DV...   
52    SanDisk 250GB Extreme Portable External SSD - ...   
67    Logitech K400 Plus Wireless Touch With Easy Me...   
...                                                 ...   
3419  Logitech B100 Corded Mouse, Wired USB Mouse fo...   
3433  INLAND Micro Center SuperSpeed 2 Pack 128GB US...   
3435  Polk Audio Signa S3 Ultra-Slim TV Sound Bar an...   
3456  Hosa YRA-167 Right Angle 3.5 mm TRS to Dual RC...   
3477  Corsair Vengeance LPX 128GB (4x32GB) DDR4 3200...   

                                            description  \
49                                                   []   
50                                                   []   
51                                                   []   
52    [The ruggedized SanDisk Extreme Portable SSD d...   
67    [The Logitech Wireless Touch Keyboard K400 Plu...   
...                                                 ...   
3419  [The B100 Optical Combo mouse. An office basic...   
3433                                                 []   
3435  [The low-profile Signa S3 sound bar delivers a...   
3456  [The YRA-167 cable is designed to split a ster...   
3477                            [Corsair Vengeance LPX]   

                                             categories   price  
49    [Electronics, Home Audio, Home Audio Accessori...    5.99  
50    [Electronics, Computers & Accessories, Compute...   14.99  
51    [Electronics, Computers & Accessories, Compute...   18.69  
52    [Electronics, Computers & Accessories, Data St...    None  
67    [Electronics, Computers & Accessories, Compute...   22.99  
...                                                 ...     ...  
3419  [Electronics, Computers & Accessories, Compute...    7.64  
3433  [Electronics, Computers & Accessories, Data St...   20.99  
3435    [Electronics, Home Audio, Speakers, Sound Bars]   299.0  
3456  [Electronics, Home Audio, Home Audio Accessori...    None  
3477  [Electronics, Computers & Accessories, Compute...  279.99  

[213 rows x 9 columns]

# Persit to OLTP

In [18]:
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
schema = os.getenv("POSTGRES_OLTP_SCHEMA")

In [19]:
print(schema)

experimental


In [20]:
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [21]:
from sqlalchemy import create_engine, text
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful:", result.scalar() == 1)
except Exception as e:
    print("Connection failed:", e)


Connection successful: True


In [22]:
insert_chunk_to_oltp(full_df_train, engine, schema, args.transaction_table_name)

Ingesting chunks:   0%|          | 0/131 [00:00<?, ?it/s]

In [23]:
holdout_rating_df.to_parquet(args.holdout_fp, index=False)